In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Reshape, Activation, Conv2D, Input, \
                                    MaxPooling2D, BatchNormalization, \
                                    Flatten, Dense, Lambda, LeakyReLU, \
                                    concatenate
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

print(tf.__version__)

2.0.0


## Model

In [2]:
# the function to implement the orgnization layer (thanks to github.com/allanzelener/YAD2K)
def space_to_depth_x2(x):
    return tf.nn.space_to_depth(x, block_size=2)

def YOLOV2(num_anchors=5, num_classes=80):
    x = inputs = tf.keras.Input(shape=(416, 416, 3))
    
    # Layer 1
    x = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(x)
    x = BatchNormalization(name='norm_1')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    
    # Layer 2
    x = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(x)
    x = BatchNormalization(name='norm_2')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    
    # Layer 3
    x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(x)
    x = BatchNormalization(name='norm_3')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    # Layer 4
    x = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(x)
    x = BatchNormalization(name='norm_4')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    # Layer 5
    x = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(x)
    x = BatchNormalization(name='norm_5')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    
    # Layer 6
    x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(x)
    x = BatchNormalization(name='norm_6')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    # Layer 7
    x = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(x)
    x = BatchNormalization(name='norm_7')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    # Layer 8
    x = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(x)
    x = BatchNormalization(name='norm_8')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    
    # Layer 9
    x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(x)
    x = BatchNormalization(name='norm_9')(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Layer 10
    x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(x)
    x = BatchNormalization(name='norm_10')(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Layer 11
    x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(x)
    x = BatchNormalization(name='norm_11')(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Layer 12
    x = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(x)
    x = BatchNormalization(name='norm_12')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    # Layer 13
    x = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(x)
    x = BatchNormalization(name='norm_13')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    conv13 = x # 
    
    x = MaxPooling2D(pool_size=(2,2))(x)
    
    # Layer 14
    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(x)
    x = BatchNormalization(name='norm_14')(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Layer 15
    x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(x)
    x = BatchNormalization(name='norm_15')(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Layer 16
    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(x)
    x = BatchNormalization(name='norm_16')(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Layer 17
    x = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(x)
    x = BatchNormalization(name='norm_17')(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Layer 18 - darknet
    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(x)
    x = BatchNormalization(name='norm_18')(x)
    darknet = LeakyReLU(alpha=0.1)(x)
    
    ##################################################################################

    # Layer 19
    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(darknet)
    x = BatchNormalization(name='norm_19')(x)
    x = LeakyReLU(alpha=0.1)(x)

    # Layer 20
    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(x)
    x = BatchNormalization(name='norm_20')(x)
    conv20 = LeakyReLU(alpha=0.1)(x)
    
    # Layer 21
    conv21 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(conv13)
    conv21 = BatchNormalization(name='norm_21')(conv21)
    conv21 = LeakyReLU(alpha=0.1)(conv21)
    conv21_reshaped = Lambda(space_to_depth_x2, name='space_to_depth')(conv21) 
    
    x = concatenate([conv21_reshaped, conv20])
    
    # Layer 22
    x = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(x)
    x = BatchNormalization(name='norm_22')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    # Layer 23 - Output
    x = Conv2D(num_anchors * (num_classes + 5), (1,1), strides=(1,1), padding='same', name='conv_23')(x)
    outputs = Lambda(lambda x: tf.reshape(x, (-1, tf.shape(x)[1], tf.shape(x)[2], 
                                              num_anchors, num_classes + 5)))(x)
    
    return Model(inputs, outputs)

In [3]:
model = YOLOV2(num_classes=1)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 416, 416, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 416, 416, 32) 128         conv_1[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 416, 416, 32) 0           norm_1[0][0]                     
______________________________________________________________________________________________

## load_darknet_weights

In [4]:
def load_darknet_weights(model, weights_file):
    with open(weights_file, 'rb') as wf:
        major, minor, revision, seen = np.fromfile(wf, dtype=np.int32, count=4)
        conv_idx = 0
        num_conv = 23 # 總共有23層的卷積層
        
        for i, layer in enumerate(model.layers):
            if not layer.name.startswith('conv'):
                continue
            conv_idx = int(layer.name[5:])
            
            # BatchNormalization layer
            batch_norm = None    
            if conv_idx < num_conv:
                batch_norm = model.get_layer('norm_' + str(conv_idx)) # 取得BatchNormalization層
            print("{}/{} {}".format(
                model.name, layer.name, 'bn' if batch_norm else 'bias'), end=', ')
            
            filters = layer.filters
            kerner_size = layer.kernel_size[0]
            input_dim = layer.input_shape[-1]
            
            if batch_norm is None:
                conv_bias = np.fromfile(wf, dtype=np.float32, count=filters)
            else:
                # darknet [beta, gamma, mean, variance]
                bn_weights = np.fromfile(
                    wf, dtype=np.float32, count=4*filters)
                # tf [gamma, beta, mean, variance]
                bn_weights = bn_weights.reshape((4, filters))[[1, 0, 2, 3]]
            
            # darknet shape (out_dim, input_dim, height, width)
            conv_shape = (filters, input_dim, kerner_size, kerner_size)
            conv_weights = np.fromfile(
                wf, dtype=np.float32, count=np.product(conv_shape))
            
            # tf shape (height, width, in_dim, out_dim)
            conv_weights = conv_weights.reshape(
                conv_shape).transpose([2, 3, 1, 0])

            if batch_norm is None:
                layer.set_weights([conv_weights, conv_bias])
            else:
                layer.set_weights([conv_weights])
                batch_norm.set_weights(bn_weights)
            
            print("Completed!")

In [5]:
weights_file = './model_data/yolov2.weights'
load_darknet_weights(model, weights_file)

model/conv_1 bn, Completed!
model/conv_2 bn, Completed!
model/conv_3 bn, Completed!
model/conv_4 bn, Completed!
model/conv_5 bn, Completed!
model/conv_6 bn, Completed!
model/conv_7 bn, Completed!
model/conv_8 bn, Completed!
model/conv_9 bn, Completed!
model/conv_10 bn, Completed!
model/conv_11 bn, Completed!
model/conv_12 bn, Completed!
model/conv_13 bn, Completed!
model/conv_14 bn, Completed!
model/conv_15 bn, Completed!
model/conv_16 bn, Completed!
model/conv_17 bn, Completed!
model/conv_18 bn, Completed!
model/conv_19 bn, Completed!
model/conv_21 bn, Completed!
model/conv_20 bn, Completed!
model/conv_22 bn, Completed!
model/conv_23 bias, Completed!


In [6]:
layer = model.layers[-2]
weights = layer.get_weights()
print(weights)

[array([[[[ 1.6324505e-01,  1.1028761e-02, -8.3031710e-03, ...,
          -4.3528821e-02, -2.9398233e-02, -6.7278352e-03],
         [ 5.0851107e-01,  1.5639421e-02,  1.4129957e-02, ...,
           2.7208056e-02, -1.0999439e-02, -2.6759051e-02],
         [ 1.3652879e-01, -3.2524453e-03,  3.6156964e-02, ...,
          -1.9063070e-02, -1.5300213e-02, -2.0567816e-02],
         ...,
         [ 2.1760814e-02,  1.3657487e-02,  4.0759966e-03, ...,
          -4.8991907e-02, -3.9354715e-02,  5.3704377e-02],
         [ 1.8230497e-04,  4.2722481e-03, -6.4058485e-04, ...,
          -2.4836339e-02, -2.2992756e-02, -1.7046183e-02],
         [-1.2241791e-03, -9.2830705e-03, -6.0601713e-04, ...,
          -3.7503283e-02, -3.7547521e-02, -1.4448024e-02]]]],
      dtype=float32), array([ 0.01750153,  0.00913803, -0.14430109, -0.08590996, -2.7688012 ,
        2.4115305 ,  0.13297267,  0.8465823 , -0.07615986, -0.26251563,
       -0.22682115, -0.3024409 ,  0.00727828,  0.0496687 ,  0.03017936,
       -0.24